Imports

In [15]:
from PIL import Image
import numpy as np
import math
from pylfsr import LFSR

## Helper Functions

- Any type to Binary Converter


In [16]:
def convertToBinary(data):
    if isinstance(data, str):
        return ''.join([ format(ord(i), "08b") for i in data ])
    elif isinstance(data, bytes) or isinstance(data, np.ndarray):
        return [ format(i, "08b") for i in data ]
    elif isinstance(data, int) or isinstance(data, np.uint8):
        return format(data, "08b")
    else:
        raise TypeError("Type not supported.")

- Binary to Decimal Converter

In [17]:
def binaryToDecimal(binary):
    decimal = 0
    for bit in binary:
        decimal = decimal*2 + int(bit)
    return decimal

- Convert image to matrix

In [18]:
def getImageMatrix(imageName):

    imageHandler = Image.open(imageName)
    pixels = imageHandler.load()
    color = 1
    if type(pixels[0,0]) == int:
        color = 0
    image_size = imageHandler.size
    
    rows = int(image_size[0])
    cols = int(image_size[1])
    image_matrix = []
    for row in range(rows):
        current_row = []
        for col in range(cols):
            current_row.append(pixels[row,col])
        image_matrix.append(current_row)
    
    return image_matrix, image_size, color


In [19]:
img, size, color = getImageMatrix("images/img1.jpeg")
print(size)

(1000, 667)


## 1D Chaotic Map

- Logistic Map

In [20]:
def LogisticMap(dimension, key, k):
    
    x = key
    a = 3.6

    bitSequence = []
    matrix = []

    for i in range(dimension * k):
        x_next = a * x * (1 - x) 
        x = x_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % k == k - 1:
            decimal = binaryToDecimal(bitSequence)
            matrix.append(decimal)
            bitSequence = []

    return matrix

## 2D Chaotic Map

- Hennon Map Implementation

In [21]:
def HennonMap(dimension, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = dimension * dimension * k 
    bitSequenceSize = k
    byteArraySize = dimension * k
    # Each bitSequence contains k bits
    bitSequence = []    
    # Each byteArray contains m bitSequence
    byteArray = []      
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            decimal = binaryToDecimal(bitSequence)
            byteArray.append(decimal)
            bitSequence = []
        
        # If Byte Sequence has k bytes, add it to Matrix.
        if i % byteArraySize == byteArraySize - 1:
            matrix.append(byteArray)
            byteArray = []

    return matrix

In [22]:
h = HennonMap(4, [0.1,0.1], 3)
print(h)

[[5, 2, 1, 6], [4, 2, 4, 7], [2, 1, 2, 3], [7, 7, 5, 2]]


- Hennon Map (Binary)

In [23]:
def HennonMapBinary(dimension, key):

    rows = dimension[0]
    cols = dimension[1]
    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    # Total Number of bitSequence produced
    sequenceSize = rows * cols
    bitSequenceSize = cols
    # Each bitSequence contains k bits
    bitSequence = []       
    # Each matrix contains m*n byteArray
    matrix = []

    for i in range(sequenceSize):
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = 0
        else:
            bit = 1

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Byte Sequence.
        if i % bitSequenceSize == bitSequenceSize - 1:
            matrix.append(bitSequence)
            bitSequence = []

    return matrix

In [24]:
h = HennonMapBinary([3,4], [0.1,0.2])
print(h)

[[1, 0, 0, 1], [0, 1, 0, 1], [0, 1, 1, 1]]


- Hennon Map (Indexes Only)

In [25]:
def HennonMapNew(required, key, k):

    x = key[0]
    y = key[1]
    a = 1.4
    b = 0.3

    chosen_pixels = {}
    chosen_pixels_count = 0

    bitSequenceSize = k
    # Each bitSequence contains 8 bits
    bitSequence = []
    indices = []

    while True:
        x_next = y + 1 - (a * (x**2)) 
        y_next = b * x
        x = x_next
        y = y_next
        if x <= 0.4:
            bit = "0"
        else:
            bit = "1"

        bitSequence.append(bit) 

        # If Bit Sequence has k bits, convert it to decimal and add it to Indices.
        if len(bitSequence) == bitSequenceSize:
            indices.append("".join(bitSequence))
            bitSequence = []
        
        # If Indices Sequence has 2 values, add it to chosen pixels.
        if len(indices) == 2:
            if (indices[0],indices[1]) not in chosen_pixels:
                chosen_pixels[(indices[0],indices[1])] = 1
                chosen_pixels_count += 1
                if chosen_pixels_count == required:
                    return chosen_pixels
            indices = []
            
    return -1

In [26]:
print(HennonMapNew(5,[0.1,0.1],3))

{('101', '010'): 1, ('001', '110'): 1, ('100', '010'): 1, ('100', '111'): 1, ('010', '001'): 1}


## Encoding


In [27]:
def encode(imageName, secret_data, key, poly):

    # read the image
    imagehandler = Image.open(imageName) 
    image_size = imagehandler.size

    # convert data to binary
    binary_secret_data = convertToBinary(secret_data)
    data_len = len(binary_secret_data)

    # Build chaotic map and derive some available pixels
    map = HennonMapBinary(image_size, key)

    availablePixels = []

    for i in range(image_size[0]):
        for j in range(0,image_size[1]):
            availablePixels.append([i,j,0])

    # check if encoding is possible
    if data_len > len(availablePixels)*6:
        raise ValueError("[!] Insufficient space. Please change key or data.")

    # Build LFSR
    lfsr = LFSR(initstate = [0, 0, 0, 0, 0, 0], fpoly=poly, counter_start_zero=True)

    for i in range(0, data_len, 2):

        index = lfsr.state
        lfsr.next()

        while availablePixels[index][2] >= 2:
            index = lfsr.state
            lfsr.next()

        row, col, usageState = availablePixels[index][0], availablePixels[index][1], availablePixels[index][2]
        pixel = imagehandler[row][col]
        r, g, b = convertToBinary(pixel)

        print(pixel)

        

        currentBits = [int(binary_secret_data[i]), int(binary_secret_data[i+1])]




    # for row in imagehandler:
    #     for pixel in row:

    #         # convert RGB values to binary format
    #         r, g, b = convertToBinary(pixel)

    #         # modify the least significant bit only if there is still data to store
    #         if data_index < data_len:
    #             # least significant red pixel bit
    #             pixel[0] = int(r[:-1] + binary_secret_data[data_index], 2)
    #             data_index += 1
    #         if data_index < data_len:
    #             # least significant green pixel bit
    #             pixel[1] = int(g[:-1] + binary_secret_data[data_index], 2)
    #             data_index += 1
    #         if data_index < data_len:
    #             # least significant blue pixel bit
    #             pixel[2] = int(b[:-1] + binary_secret_data[data_index], 2)
    #             data_index += 1
    #         # if data is encoded, just break out of the loop
    #         if data_index >= data_len:
    #             break
    # return image

In [29]:
encode("images/img1.jpeg", "data here", [0.1, 0.1], [1,0,1])

ValueError: Invalid feedback polynomial: Order of feedback polynomial can not be less than 2 or greater than length of state vector. 
 Polynomial also can not have negative or zeros powers